# データの統合とデータクレンジング

## データの統合

In [1]:
# 標準モジュール読み込み
import pandas as pd

In [2]:
df = pd.DataFrame(columns=["cd-num", "haiku", "writer", "kigo", "season"])

In [3]:
for j in range(500):
    unit = j * 100
    add_df = pd.read_csv(str(unit)+'.csv')
    df = df.append(add_df)

In [4]:
df.to_csv("marged_raw.csv")

## データクレンジング（null値の削除）

In [5]:
# サイズ
df.shape

(50000, 5)

In [6]:
# nullの数
df.isnull().sum()

cd-num        0
haiku     10839
writer    10839
kigo      11769
season    10885
dtype: int64

In [7]:
# 今回は季語がないものが多く含まれるようだ。とりあえず、(haiku/writer/kigo/season)のいづれかがnullっているものを削除する。
df_delete_null = df.dropna(subset=['haiku','writer','kigo','season'])

In [8]:
df_delete_null.shape

(38188, 5)

In [9]:
df_delete_null.head()

,cd-num,haiku,writer,kigo,season
1,1,朝霧の中に九段のともし哉,正岡子規,霧,秋
2,2,あたたかな雨が降るなり枯葎,正岡子規,枯葎,冬
3,3,菜の花やはつと明るき町はづれ,正岡子規,菜の花,春
4,4,秋風や伊予へ流るる汐の音,正岡子規,秋風,秋
5,5,長閑さや障子の穴に海見えて,正岡子規,長閑,春


In [10]:
# 確認
df_delete_null.isnull().sum()

cd-num    0
haiku     0
writer    0
kigo      0
season    0
dtype: int64

In [11]:
df_delete_null.to_csv("df_delete_null.csv")

## データクレンジング（（）：ルビや/：句切れなど記号を削除）

In [12]:
# （）()... 全角半角の括弧の組み合わせで正規表現でチェック
df_delete_null[df_delete_null["haiku"].str.contains("(\(|（).+?(\)|）)")].head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


,cd-num,haiku,writer,kigo,season
24,24,漱石が来て虚子が来て大三十日（おほみそか）,正岡子規,大晦日,冬
43,43,つり鐘の蔕（へた）のところが渋かりき,正岡子規,柿,秋
44,44,三千の俳句を閲（けみ）し柿二つ,正岡子規,柿,秋
46,46,この頃の蕣（あさがほ）藍に定まりぬ,正岡子規,朝顔,秋（夏）
56,56,薔薇の香の紛々（ふんぷん）として眠られず,正岡子規,薔薇,夏


In [13]:
df_cleaning = df_delete_null.replace("(\(|（).+?(\)|）)","", regex=True)

In [14]:
df_cleaning.shape

(38188, 5)

In [15]:
# /などの句切れをチェック
df_cleaning[df_cleaning["haiku"].str.contains("／|\/|\\|＼")].head()

,cd-num,haiku,writer,kigo,season
50,3150,身をそらす虹の/絶巓/処刑台,高柳重信,虹,夏
51,3151,「月光」旅館/開けても開けてもドアがある,高柳重信,月光,秋
52,3152,月下の宿帳/先客の名はリラダン伯爵,高柳重信,月光,秋
56,3156,軍鼓鳴り/荒涼と/秋の/痣となる,高柳重信,秋,秋
64,3164,後朝や/いづこも/伊豆の/神無月,高柳重信,神無月,冬


In [16]:
df_cleaning2 = df_cleaning.replace("／|\/|\\|＼","", regex=True)

In [17]:
# 最後にインデックス番号を付け直し
df_cleaning3 = df_cleaning2.reset_index(drop = True)

In [18]:
df_cleaning3.to_csv("df_cleaning3.csv")